# Segmenting and Clustering Neighborhoods In Toronto

## Project's description
This project includes scraping the Wikipedia page and wrangling the required data, and then processing, cleaning and reading that data into a pandas structured formate dataframe which is later used to explore, segment and cluster the neighborhoods in the city of Toronto, Canada. The clustering is carried out by K-Means.
### All tasks (*web scraping, wrangling, data cleaning, exploring and clustering the neighborhoods*) are implemented in the same notebook for the ease of evalution.

## Installing / Importing the required libraries

In [1]:
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from bs4 import BeautifulSoup

#!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
from IPython.display import display_html
    
import requests # library to handle requests
from pandas.io.json import json_normalize # tranforming json file into a pandas dataframe library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes
print('Folium installed')
import folium # plotting library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    folium-0.5.0               |             py_0          45 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    branca-0.4.1               |             py_0          26 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                       

## 1. Data scrapping, wrangling, pre-processing and cleaning

### 1.1 Scraping the Wiki page, Getting the data and Converting into pandas dataframe

Scraping the wiki page

In [68]:
# getting data from internet
dataSourceLink = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
rawData = requests.get(dataSourceLink).text

soup = BeautifulSoup(rawData,'lxml') # parse the HTML/XML codes by using beautiful soup

Get the data from the HTML page and store it into a list

In [69]:
# using soup object, iterate the wikitable
dataTable = soup.find('table')
row = []  #for rows
columnsName = []  # to store headers

for index, tr in enumerate(dataTable.find_all('tr')):
    results = []
    for td in tr.find_all(['th', 'td']):
        results.append(td.text.rstrip())
        
    # as first row of dataTable is the header (title for columns) and rest are rows
    if (index == 0):
        columnsName = results
    else:
        row.append(results)  

Convert into pandas Dataframe

In [70]:
df_toronto = pd.DataFrame(data = row, columns = columnsName)
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


### 1.2 Data pre-processing and cleaning 

Remove the rows where Borough is "Not assigned"

In [71]:
df_toronto = df_toronto[df_toronto['Borough'] != 'Not assigned']
df_toronto.head()

,Postal Code,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


Combine the neighborhoods with the same Postal Code

In [72]:
df_toronto = df_toronto.groupby(["Postal Code", "Borough"])["Neighborhood"].apply(", ".join).reset_index()
df_toronto.head()

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Replace the name of the neighborhoods which are 'Not assigned' with the names of Borough

In [73]:
df_toronto["Neighborhood"].replace("Not assigned", df_toronto["Borough"], inplace = True)
df_toronto.head(10)

,Postal Code,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park"
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge"
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


### 1.3 Print the shape of Dataframe

Print the number of rows of the dataframe by using .shape function.

In [74]:
print("Shape of DataFrame df_toronto is :", df_toronto.shape)
print("The total number of rows of dataframe df_toronto are ", df_toronto.shape[0])

Shape of DataFrame df_toronto is : (103, 3)
The total number of rows of dataframe df_toronto are  103


## 2. Importing and Merging the Geospatial Data

Adding the geographical coordinates of the neighborhoods using given csv file

In [75]:
df_toronto_lat_lon = pd.read_csv('https://cocl.us/Geospatial_data')
df_toronto_lat_lon.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge the two dataframes (df_toronto & df_toronto_lat_lon) into one dataframe

In [76]:
df_toronto = pd.merge(df_toronto, df_toronto_lat_lon, on = 'Postal Code')
df_toronto.head(15)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Kennedy Park, Ionview, East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile, Clairlea, Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside, Cliffcrest, Scarborough Village West",43.716316,-79.239476
9,M1N,Scarborough,"Birch Cliff, Cliffside West",43.692657,-79.264848


## 3. Exploring and Clustering the neighborhoods in Toronto
This section includes mainly 3 sub-sections and each section again includes sub-sub-sections:
* Filtering data and Visualizing all the neighborhoods in a part of Toronto city (2 sub-sub-sections)
* Exploring using Four Square API (4 sub-sub-sections)
* Clustering Neighborhoods using K-Means clustering and Visualization (3 sub-sub-sections)

### 3.1 Filtering data and Visualizing all the neighborhoods in a part of Toronto city

### 3.1.1 Getting all the Neighborhoods which contain Toronto in their Borough

In [77]:
df_onlyTorontoBorough = df_toronto[df_toronto['Borough'].str.contains("Toronto")].reset_index(drop=True)
print("Total number of Boroughs which contain Toronto are", df_onlyTorontoBorough.shape[0])
df_onlyTorontoBorough.head()

Total number of Boroughs which contain Toronto are 39


,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790


### 3.1.2 Visulaizing these all Neighborhoods using Folium

In [78]:
# Get the latitude and longitude values of Toronto by using Geopy library
address = "Toronto, ON"
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto city are {}, {}.'.format(latitude, longitude))

# Create map of Toronto using latitude and londitude values (we could also use the first entried latitude and longitude values from the dataframe)
map_toronto = folium.Map(location = [latitude, longitude], zoom_start = 10)

# add also marker to map
for lat, lng, borough, neighborhood in zip(df_onlyTorontoBorough['Latitude'],
                                            df_onlyTorontoBorough['Longitude'],
                                            df_onlyTorontoBorough['Borough'],
                                            df_onlyTorontoBorough['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
    [lat,lng],
    radius=5,
    popup=label,
    color='blue',
    fill=True,
    fill_color='#3186cc',
    fill_opacity=0.7,
    parse_html=False).add_to(map_toronto)

map_toronto

The geograpical coordinate of Toronto city are 43.6534817, -79.3839347.


### 3.2 Exploring using Four Square API

### 3.2.1 Defining Foursquare Credentials and Version
Due to privacy reason, It will be not visible on the public repository on Github.

In [79]:
# The code was removed by Watson Studio for sharing.

### 3.2.2 Exploring the first Neighborhood from the dataframe "df_onlyTorontoBorough"

**Getting the name and latitude and longitude values of the first Neighborhood**

In [80]:
nbrhd_name = df_onlyTorontoBorough['Neighborhood'][0]
nbrhd_lat = df_onlyTorontoBorough['Latitude'][0]
nbrhd_lng = df_onlyTorontoBorough['Longitude'][0]

print(f"The first neighborhood is '{nbrhd_name}' and the values of its latitude and longitude are ",
     nbrhd_lat, "and", nbrhd_lng, ", respectively.")

The first neighborhood is 'The Beaches' and the values of its latitude and longitude are  43.67635739999999 and -79.2930312 , respectively.


**Now, get the top 100 venues that are in "The Beaches" within a radius of 500 meters.**

In [81]:
# set up API url to explore the first neighborhood place
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, CLIENT_SECRET, VERSION, nbrhd_lat, nbrhd_lng, radius, LIMIT)
url

# Get the results
nbrhd_results = requests.get(url).json()
 

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
# Clean the json and structure it into a pandas dataframe.
venues = nbrhd_results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

#filter colums
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))
nearby_venues.head()

4 venues were returned by Foursquare.


,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


### 3.2.3 Exploring the Neighborhoods in a part of Toronto City

**Create a function to repeat the same process to all the neighborhoods in "df_onlyTorontoBorough" dataframe**

In [82]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        nbrhd_results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in nbrhd_results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

**Get Venues data for all Neighborhoods and store it in a new dataframe called 'toronto_nearby_venues'**

In [83]:
toronto_nearby_venues = getNearbyVenues(names = df_onlyTorontoBorough['Neighborhood'],
                                        latitudes = df_onlyTorontoBorough['Latitude'],
                                        longitudes = df_onlyTorontoBorough['Longitude'] )

print('\n\nTotal {} nearby venues were returned by Foursquare.'.format(toronto_nearby_venues.shape[0]))
toronto_nearby_venues.head()

The Beaches
The Danforth West, Riverdale
India Bazaar, The Beaches West
Studio District
Lawrence Park
Davisville North
North Toronto West,  Lawrence Park
Davisville
Moore Park, Summerhill East
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North & West, Forest Hill Road Park
The Annex, North Midtown, Yorkville
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
Stn A PO Boxes
First Canadian Place, Underground city
Christie
Dufferin, Dovercourt Village
Little Portugal, Trinity
Brockton, Parkdale Village, Exhibition Place
High

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


**Check the total nearby venues for each neighborhood**

In [84]:
toronto_nearby_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,58,58,58,58,58,58
"Brockton, Parkdale Village, Exhibition Place",23,23,23,23,23,23
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",18,18,18,18,18,18
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",14,14,14,14,14,14
Central Bay Street,70,70,70,70,70,70
Christie,16,16,16,16,16,16
Church and Wellesley,71,71,71,71,71,71
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [85]:
# Unique categories from all the returned venues
print('There are {} uniques categories.'.format(len(toronto_nearby_venues['Venue Category'].unique())))

There are 232 uniques categories.


### 3.2.4 Analyze Each Neighborhood

In [86]:
# one hot encoding
toronto_nearby_onehot = pd.get_dummies(toronto_nearby_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_nearby_onehot['Neighborhood'] = toronto_nearby_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_nearby_onehot.columns[-1]] + list(toronto_nearby_onehot.columns[:-1])
toronto_nearby_onehot = toronto_nearby_onehot[fixed_columns]

toronto_nearby_onehot.head()

,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,Art Gallery,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


**Grouping of rows by neighborhood and by taking the mean of the frquency of occurrence of each category**

In [87]:
toronto_nearby_grouped = toronto_nearby_onehot.groupby('Neighborhood').mean().reset_index()
toronto_nearby_grouped.head()

,Neighborhood,Yoga Studio,Airport,Airport Food Court,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,Aquarium,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wine Shop,Women's Store
0,Berczy Park,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.017241,0.0,0.0,0.000000,0.0,0.0
1,"Brockton, Parkdale Village, Exhibition Place",0.043478,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
2,"Business reply mail Processing Centre, South C...",0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
3,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.071429,0.071429,0.071429,0.214286,0.071429,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0
4,Central Bay Street,0.014286,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.014286,0.0,0.0,0.014286,0.0,0.0


**Write the function to sort the venues in descending order and display the top 10 venues for each neighborhood**

In [88]:
# function
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#10 top most common venues
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_nearby_grouped['Neighborhood']

for ind in np.arange(toronto_nearby_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_nearby_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Berczy Park,Coffee Shop,Cocktail Bar,Cheese Shop,Café,Restaurant,Pharmacy,Bakery,Farmers Market,Beer Bar,Seafood Restaurant
1,"Brockton, Parkdale Village, Exhibition Place",Café,Breakfast Spot,Coffee Shop,Yoga Studio,Bakery,Stadium,Burrito Place,Restaurant,Climbing Gym,Pet Store
2,"Business reply mail Processing Centre, South C...",Light Rail Station,Garden,Brewery,Burrito Place,Spa,Farmers Market,Fast Food Restaurant,Butcher,Restaurant,Recording Studio
3,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Service,Harbor / Marina,Boat or Ferry,Coffee Shop,Plane,Rental Car Location,Sculpture Garden,Boutique,Airport Terminal,Airport
4,Central Bay Street,Coffee Shop,Sandwich Place,Italian Restaurant,Café,Japanese Restaurant,Bubble Tea Shop,Burger Joint,Thai Restaurant,Bar,Salad Place


### 3.3 Clustering Neighborhoods using K-Means clustering and Visualization

### 3.3.1 K-Means Clustering

In [89]:
# set nubmer of clusters
k_clusters = 5

toronto_grouped_clustering = toronto_nearby_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters = k_clusters, random_state = 0).fit(toronto_grouped_clustering)

print('Generated cluster labels', kmeans.labels_)

# add the clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_nearby_merged = df_onlyTorontoBorough

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_nearby_merged = toronto_nearby_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

toronto_nearby_merged.head() # check the last columns!

Generated cluster labels [0 0 0 0 0 0 0 0 0 0 0 0 2 0 0 0 0 0 3 0 1 0 0 0 0 0 1 4 0 0 0 0 0 0 0 0 0
 0 0]


,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,0,Health Food Store,Trail,Pub,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Liquor Store,Spa,Juice Bar
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572,0,Park,Pub,Sandwich Place,Burrito Place,Board Shop,Restaurant,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,M4M,East Toronto,Studio District,43.659526,-79.340923,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Convenience Store,Sandwich Place,Cheese Shop,Clothing Store
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,3,Park,Bus Line,Swim School,Women's Store,Dessert Shop,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop


### 3.3.2 Visualization of the resulting clusters

In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_clusters)
ys = [i + x + (i*x)**2 for i in range(k_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(
        toronto_nearby_merged['Latitude'], 
        toronto_nearby_merged['Longitude'], 
        toronto_nearby_merged['Neighborhood'], 
        toronto_nearby_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### 3.3.3 Examine Cluster 1

In [91]:
toronto_nearby_merged.loc[toronto_nearby_merged['Cluster Labels'] == 0,
                          toronto_nearby_merged.columns[[1] + list(range(5, toronto_nearby_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,0,Health Food Store,Trail,Pub,Women's Store,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Donut Shop
1,East Toronto,0,Greek Restaurant,Italian Restaurant,Coffee Shop,Restaurant,Ice Cream Shop,Furniture / Home Store,Yoga Studio,Liquor Store,Spa,Juice Bar
2,East Toronto,0,Park,Pub,Sandwich Place,Burrito Place,Board Shop,Restaurant,Fast Food Restaurant,Fish & Chips Shop,Italian Restaurant,Steakhouse
3,East Toronto,0,Café,Coffee Shop,Brewery,Gastropub,Bakery,American Restaurant,Convenience Store,Sandwich Place,Cheese Shop,Clothing Store
5,Central Toronto,0,Hotel,Department Store,Gym / Fitness Center,Park,Pizza Place,Breakfast Spot,Sandwich Place,Food & Drink Shop,Dog Run,Diner
6,Central Toronto,0,Clothing Store,Sporting Goods Shop,Coffee Shop,Gym / Fitness Center,Fast Food Restaurant,Diner,Mexican Restaurant,Park,Pet Store,Chinese Restaurant
7,Central Toronto,0,Sandwich Place,Dessert Shop,Coffee Shop,Gym,Café,Italian Restaurant,Sushi Restaurant,Pizza Place,Pharmacy,Seafood Restaurant
9,Central Toronto,0,Pub,Coffee Shop,Bagel Shop,Liquor Store,Restaurant,Sports Bar,Bank,Supermarket,Sushi Restaurant,Fried Chicken Joint
11,Downtown Toronto,0,Coffee Shop,Restaurant,Café,Bakery,Pub,Pizza Place,Italian Restaurant,Playground,Pet Store,Pharmacy
12,Downtown Toronto,0,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant,Yoga Studio,Mediterranean Restaurant,Pub,Hotel,Men's Store
